In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
from datetime import datetime
from datetime import date
from IPython.display import HTML
requests.packages.urllib3.disable_warnings()
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
snapshot_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"

In [4]:
snapshot = requests.get(snapshot_url)
snapshot

<Response [200]>

In [5]:
def extract_data(product):
    title = product.select_one('.wd-entities-title').text
    
    price_elements = product.select('.woocommerce-Price-amount')
    regular_price = price_elements[0].text
    # tmp_reg = regular_price.split()
    # regular_price = tmp_reg[0]

    if len(price_elements) > 1:
        discount_price = price_elements[1].text
        # tmp_disc = discount_price.split()
        # discount_price = tmp_disc[0]
    else:
        discount_price = None

    product_url = product.select_one('.wd-entities-title a')['href']

    add_to_cart_url = product.select_one('.add_to_cart_button')['href']

    return {'Title': title, 'Regular Price': regular_price, 'Discount Price': discount_price,
            'Product URL': product_url, 'Add to Cart URL': add_to_cart_url}

In [6]:
products = []

page = 1

while True:
    # print(f"Page: {page}.")
    
    page_url = f"{snapshot_url}page/{page}/"
    response = requests.get(page_url)
    
    if response.status_code != 200:
        print("No more pages.")
        break
    
    soup = BeautifulSoup(response.content, "html.parser")
    product_items = soup.select('.product')
    
    if not product_items:
        print("No more products.")
        break
    
    for product in product_items:
        product_data = extract_data(product)
        products.append(product_data)

    page += 1
# print("Complete!")

No more pages.


In [23]:
df = pd.DataFrame(products)
df

,Title,Regular Price,Discount Price,Product URL,Add to Cart URL
0,Acer A315-23-A7KD,17.590 ден,None,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490 ден,None,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999 ден,15.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490 ден,None,https://clevershop.mk/product/21498/,?add-to-cart=21498
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12.890 ден,None,https://clevershop.mk/product/monitor-27-phili...,?add-to-cart=12618
316,Philips 24″ 243V7QDSB,8.390 ден,None,https://clevershop.mk/product/philips-24%e2%80...,?add-to-cart=12396
317,Philips 27″ 278E1A/00 4K UHD IPS,18.990 ден,None,https://clevershop.mk/product/hp-27%e2%80%b3-2...,?add-to-cart=12218
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26.990 ден,None,https://clevershop.mk/product/philips-279c9-00...,?add-to-cart=12578


In [7]:
df.to_csv("laboratoriska1_csv_file.csv", sep=';', index=False)